### Libraries import

In [27]:
import pandas as pd # Dataframes management
from zipfile import ZipFile  # Files compressed management
import os # Files management along OS
import re

In [28]:
from os import mkdir
from os.path import exists

In [29]:
from selenium import webdriver # Webscrapping bot
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

### Modules

In [30]:
from db import *

### Data import

In [31]:
# Specifying the name of the zip file
file = "../Inputs/Modificados - Atmira_Pharma_Visualization/items_ordered_2years_V2.zip"
  
# Open the zip file in read mode
with ZipFile(file, 'r') as zip: 
    # List all the contents of the zip file
    zip.printdir() 
  
    # Extract all files
    print('extraction...') 
    zip.extractall("../Inputs/Modificados - Atmira_Pharma_Visualization") 
    print('Done!')

#Import CSV to pandas
itemsOrdered = pd.read_csv("../Inputs/Modificados - Atmira_Pharma_Visualization/items_ordered_2years_V2.csv")
print("CSV imported to Pandas successfully")

# Remove uncompressed CSV file
os.remove("../Inputs/Modificados - Atmira_Pharma_Visualization/items_ordered_2years_V2.csv")
print("Original CSV removed to preserve repo health")

File Name                                             Modified             Size
items_ordered_2years_V2.csv                    2022-02-23 01:28:04    150372498
extraction...
Done!
CSV imported to Pandas successfully
Original CSV removed to preserve repo health


In [32]:
itemsOrdered.head(3)

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode
0,562ba4ba9aaf2a1e926842deca19271d,2bca87c2ab10b07feef290a9be47f316,2017-01-01 00:20:33,6409.0,1.0,13.4204,17.17,8.0,50d7f99947b472cc889d58845b9d23e2,Valencia,46019
1,562ba4ba9aaf2a1e926842deca19271d,53e12ac4b579f0c4b1c4f77d50bf8792,2017-01-01 00:20:33,5133.0,1.0,18.5351,22.68,8.0,50d7f99947b472cc889d58845b9d23e2,Valencia,46019
2,562ba4ba9aaf2a1e926842deca19271d,d19d6df5cd6a1e741408b70aa2767bf4,2017-01-01 00:20:33,5125.0,1.0,18.5260,22.68,8.0,50d7f99947b472cc889d58845b9d23e2,Valencia,46019


In [33]:
del zip

### Arreglos para facilitar el webscrapping

In [34]:
itemsOrdered["city"].replace(to_replace={'San Vicente del Raspeig':'Sant Vicent del Raspeig'}, inplace=True)

In [35]:
itemsOrdered.loc[itemsOrdered['zipcode'].eq('30139') & itemsOrdered['city'].eq('Murcia'), "city"] = "EL RAAL"

In [36]:
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("29039", "28039")

In [37]:
itemsOrdered['zipcode'] = itemsOrdered['zipcode'].replace("29039", "28039")

In [38]:
# itemsOrdered["city"].replace(to_replace={'Pamplona':'Pamplona/Iruna'}, inplace=True)

In [39]:
# itemsOrdered["city"].replace(to_replace={'Aravaca-Madrid':'Madrid'}, inplace=True)

In [40]:
itemsOrdered.loc[itemsOrdered['zipcode'].eq('33195') & itemsOrdered['city'].eq('Oviedo '), "city"] = "Morente"

### USO DE WEBSCRAPPING PARA CÓDIGO POSTAL

Creación de Base de Datos para almacenamiento de resultados formateados del WebScrapping

In [41]:
def IntroDB():
    try:
        pathDB = "../databases/"
        nameDB= "scrapedZips.db"
        if exists(pathDB):
            print(f"Carpeta {pathDB} encontrada")
        else:
            mkdir(pathDB)
            print (f"Creada carpeta {pathDB}")
    except OSError:
        print (f"Creación de carpeta {pathDB} falló")
        
    if exists(f"{pathDB}{nameDB}"):
        print("Database existe")
        con, cur = SqlConnection(f"{pathDB}{nameDB}")
        return con, cur
    else:
        print("Database no existe")
        CreateCon(f"{pathDB}{nameDB}")
        con, cur = SqlConnection(f"{pathDB}{nameDB}")
        PrepareCon(con, cur, option="insert",
            values=("Test","Test","Test","Test"))
        return con, cur


In [42]:
con, cur = IntroDB()

Carpeta ../databases/ encontrada
Database existe
Conexión establecida


In [43]:
try:
    if GetThings(cur, selection="Country,Region,City,Zipcode", where=["ID", 1], limit=1)[0][0] == "Test":
        PrepareCon(con,cur,where=["ID",1],option="delete")
        cur.execute("UPDATE `sqlite_sequence` SET `seq` = 0 WHERE `name` = 'ZipsInfo';")
except IndexError:
    pass

'''
Check the next query if something perturbe your mind
SELECT * FROM `sqlite_sequence`;
'''

'\nCheck the next query if something perturbe your mind\nSELECT * FROM `sqlite_sequence`;\n'

In [44]:
GetThings(cur, selection="Country,Region,City,Zipcode", where=["ID", 1], limit=1)

[('Spain', 'Comunidad Valenciana', 'Valencia', '46019')]

A continuación se genera una lista compuesta de tuplas compuestas de la siguiente forma: ("Ciudad", "Zipcode")

In [45]:
rawDataZipcode = list(zip(itemsOrdered["city"].tolist(), itemsOrdered["zipcode"].tolist()))
rawDataZipcode = rawDataZipcode[:int(len(rawDataZipcode)/30)]

In [46]:
rawDataZipcode[0]

('Valencia', '46019')

**Funciones destacadas**

Función que limpia zipcodes

In [47]:
#Limpieza de zipcodes con RegEx
def num_guion(string):
    """ Get a string with the numbers and hyphens of another string
    
    Args:
        df: string used to extract the string with numbers abd hyphens

    Returns:
        df: the string with numbers and hyphens
    """
    aux = re.match("([\d-]+)", str(string))
    try:
        return str(aux.group())
    except:
        return string


itemsOrdered["zipcode"] = itemsOrdered["zipcode"].apply(lambda x: num_guion(x))

Función que limpia los acentos con el fin de homogeneizar

In [48]:
def AcentosLimpiador(text):
	acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'Á': 'A', 'E': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}
	for ele in acentos:
		if ele in text:
			text = text.replace(ele, acentos[ele])
	return text

Función que formatea los resultados del webscrapping de forma adecuada a los requerimientos necesarios

In [49]:
def zipCodeManipulation(city, zipcode, queryResult="", saved = False):
  
    # This conditional checks if the zipcode info was scrapped and stored succesfully
    if saved == False:

        listTestingZipcode = queryResult.split("\n")
        indexMatchRegex = list(map(lambda x: [(m.start(0), m.end(0)) for m in re.finditer(r"[a-z][A-Z0-9]", x)], listTestingZipcode))

        resultScrapClean = []
        for pas,resultScrap in enumerate(listTestingZipcode):
            for pos,ele in enumerate(indexMatchRegex[pas]):
                if len(indexMatchRegex[pas])==2:
                    if pos ==0:
                        txt = resultScrap[:ele[0]+1]+","+resultScrap[ele[1]-1:]
                    elif pos ==1:
                        txt = txt[:ele[0]+2]+","+txt[ele[1]:]
                        resultScrapClean.append(txt)
                elif len(indexMatchRegex[pas])==3:
                    if pos ==0:
                        txt = resultScrap[:ele[0]+1]+","+resultScrap[ele[1]-1:]
                    elif pos ==1:
                        txt = txt[:ele[0]+2]+","+txt[ele[1]:]
                    elif pos ==2:
                        txt = txt[:ele[0]+3]+","+txt[ele[1]+1:]
                        resultScrapClean.append(txt)

        resultScrapListed = [element.split(",") for element in resultScrapClean]
        resultScrapRearr = [(element[0], element[1], element[-2], element[-1]) for element in resultScrapListed]

        resultZip = []
        for element in resultScrapRearr:
            if (AcentosLimpiador(element[2].lower()).rstrip().__contains__(AcentosLimpiador(city.lower()).rstrip())):
                resultZip.append(element)
                break   
        resultZip = resultZip[0]

    elif saved == True:
        resultZip = GetThings(cur, selection="Country,Region,City,Zipcode", where=["Zipcode", zipcode], limit=1, simplify = True)

    return resultZip

**Webscrapping!**

In [50]:
try:
    
    driver = webdriver.Firefox()
    
    driver.get("https://worldpostalcode.com/lookup")
    driver.set_page_load_timeout(5)

    for element in rawDataZipcode:

        if GetThings(cur, selection="Zipcode", where=["Zipcode", element[1]], limit=1, simplify=True) == None:
            insertZipcode =driver.find_element(By.ID,"search")
            insertZipcode.clear()
            insertZipcode.send_keys(element[1])
            clickButtonZipcode =driver.find_element(By.CLASS_NAME,"submit")
            clickButtonZipcode.click()
            driver.set_page_load_timeout(5)

            getGeoInfo = driver.find_element(By.CLASS_NAME,"search_units")
            saveGeoInfo = getGeoInfo.text
            zipCodeDef = zipCodeManipulation(element[0], element[1], queryResult=saveGeoInfo)
        else:
            zipCodeDef = zipCodeManipulation(city=element[0], zipcode=element[1], saved = True)

        PrepareCon(con, cur, values=zipCodeDef, option="insert", verbose=False)

    driver.close()
    con.close()

except IndexError:
    driver.close()
    raise Exception(f"El fallo se ha producido con ciudad: {element[0]}, zipcode: {element[1]}")

Exception: El fallo se ha producido con ciudad: CARIDAD, LA (EL FRANCO), zipcode: 33750

**Comprobaciones del scrapping y formateo posterior realizado**

In [ ]:
AcentosLimpiador("Requena".lower()) == AcentosLimpiador("Requena ".lower())

False

**Transformación del scrapeo formateado a dataframe de Pandas**

In [ ]:
asd = pd.DataFrame(masterScrapping, columns=["Country","Region","City","Zipcode"])
asd.head(3)

,Country,Region,City,Zipcode
